In [1]:
import yfinance as yf
import pandas as pd
import statsmodels.api as sm
import bs4 as bs
import requests
import datetime as dt
import pickle5 as pickle
from strategy.factors.beta import Beta

In [2]:
resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = bs.BeautifulSoup(resp.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})
tickers = []
for row in table.findAll('tr')[1:]:
    ticker = row.findAll('td')[0].text
    tickers.append(ticker)

tickers_list = [s.replace('\n', '') for s in tickers]

In [3]:
tickers_list

['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ACN',
 'ATVI',
 'ADM',
 'ADBE',
 'ADP',
 'AAP',
 'AES',
 'AFL',
 'A',
 'APD',
 'AKAM',
 'ALK',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AMD',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'ABC',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'AON',
 'APA',
 'AAPL',
 'AMAT',
 'APTV',
 'ACGL',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'AZO',
 'AVB',
 'AVY',
 'BKR',
 'BALL',
 'BAC',
 'BBWI',
 'BAX',
 'BDX',
 'WRB',
 'BRK.B',
 'BBY',
 'BIO',
 'TECH',
 'BIIB',
 'BLK',
 'BK',
 'BA',
 'BKNG',
 'BWA',
 'BXP',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF.B',
 'CHRW',
 'CDNS',
 'CZR',
 'CPT',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CTLT',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'CNC',
 'CNP',
 'CDAY',
 'CF',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMCSA',
 'CMA

In [4]:
start = dt.datetime(2000,1,1)
end = dt.datetime(2021,12,31)
df = yf.download(tickers_list, start=start, end=end)
sp_price = df['Adj Close']

sp_price.index = pd.to_datetime(pd.to_datetime(sp_price.index).date)
sp_price.index.name = 'date_time'

sp_price

[*********************100%***********************]  503 of 503 completed

4 Failed downloads:
- GEHC: Data doesn't exist for startDate = 946702800, endDate = 1640926800
- BF.B: AttributeError("'NoneType' object has no attribute 'tz_convert'")
- CEG: Data doesn't exist for startDate = 946702800, endDate = 1640926800
- BRK.B: No timezone found, symbol may be delisted


,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACGL,ACN,ADBE,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
date_time,,,,,,,,,,,,,,,,,,,,,
2000-01-03,44.001263,NaN,NaN,0.851942,NaN,2.998109,9.221023,1.277778,NaN,16.274673,...,NaN,7.294346,19.130630,6.772067,NaN,4.792225,NaN,25.027779,37.955738,NaN
2000-01-04,40.640045,NaN,NaN,0.780115,NaN,2.793420,8.957568,1.270833,NaN,14.909399,...,NaN,7.462313,18.764200,6.772067,NaN,4.695899,NaN,24.666668,36.117771,NaN
2000-01-05,38.119152,NaN,NaN,0.791530,NaN,3.010150,8.941098,1.388889,NaN,15.204176,...,NaN,7.750242,19.787155,6.889139,NaN,4.719979,NaN,25.138889,36.075054,NaN
2000-01-06,36.667721,NaN,NaN,0.723033,NaN,3.238922,9.253957,1.375000,NaN,15.328289,...,NaN,7.678260,20.810101,6.898145,NaN,4.679843,NaN,23.777779,36.587967,NaN
2000-01-07,39.723373,NaN,NaN,0.757282,NaN,3.660344,9.352757,1.451389,NaN,16.072987,...,NaN,7.678260,20.749035,6.880134,NaN,4.575490,NaN,23.513889,36.673447,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-23,156.562347,18.260000,224.063004,175.262802,126.892914,127.924416,136.246506,43.480000,396.261047,569.619995,...,88.879997,64.804298,58.691696,54.536751,115.991066,132.787643,122.040504,582.409973,60.688782,240.179337
2021-12-27,157.494980,18.170000,228.281128,179.289459,128.151459,130.265045,138.498352,43.930000,408.070953,577.679993,...,87.580002,65.008598,59.528496,55.048370,116.770912,135.407303,123.061127,606.330017,61.321358,244.140900
2021-12-28,157.931503,18.540001,229.854492,178.255417,128.132370,130.719345,137.529068,44.270000,408.011993,569.359985,...,86.459999,65.786911,59.336124,55.127075,117.985115,135.377853,123.446266,597.320007,61.418682,241.902634


In [5]:
sp_price = sp_price.dropna(axis=1)

In [6]:
test = Beta(price_df=sp_price, benchmark_ticker='SPY').get_beta()

[*********************100%***********************]  1 of 1 completed


In [7]:
test

,beta
A,1.153635e+00
AAPL,1.044382e+00
ABC,6.292169e-01
ABT,5.775147e-01
ACGL,6.210067e-01
...,...
YUM,7.357002e-01
ZBRA,9.340354e-01
ZION,1.180239e+00
SPY,1.000000e+00
